In [49]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
json_prepared_filtered = dataiku.Dataset("Json_prepared_filtered")
df = json_prepared_filtered.get_dataframe()

In [50]:
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd
import time

import os

In [51]:
del df['Timestamp']
df_cpy  = df.set_index('date')

In [52]:
val = df_cpy
scalers={}
for i in df_cpy.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(val[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    val[i]=s_s

In [53]:
val_to_pred = val.values
val_to_pred = val_to_pred.reshape((1, val_to_pred.shape[0],val_to_pred.shape[1]))

In [54]:
handle = dataiku.Folder("model_forecast")
json_config = handle.read_json("model_json")
model = keras.models.model_from_json(json_config)

/opt/dataiku/code-env/lib64/python3.6/site-packages/tensorflow_core/python/framework/tensor_util.py:521: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  tensor_proto.tensor_content = nparray.tostring()


In [55]:
val_pred = model.predict(val_to_pred)

In [57]:
for index,i in enumerate(df_cpy.columns):
    scaler = scalers['scaler_'+i]
    val_pred[:,:,index]=scaler.inverse_transform(val_pred[:,:,index])

In [58]:
val = val_pred.squeeze()
val.shape

(5, 30)

In [62]:
df_pred = pd.DataFrame(val, columns=df_cpy.columns)

In [63]:
lastdate = df["date"].max()
listdate = []
for i in range(len(val)):
    lastdate = lastdate + timedelta(minutes=15)
    listdate.append(lastdate)


In [0]:
df_pred["date"] = 

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

last_prediction_df = json_prepared_filtered_df # For this sample code, simply copy input to output


# Write recipe outputs
last_prediction = dataiku.Dataset("last_prediction")
last_prediction.write_with_schema(last_prediction_df)